This document is intended to be used with my paper "An Improvement on a Gap Principle Counting Technique and Some Consequences" of which you can find a preprint here: https://arxiv.org/abs/2210.09631

In [1]:
from math import sqrt,floor,log
from collections import deque
import decimal
decimal.getcontext().prec = 10**4

# Function Definitions

The function below corresponds to the quantity $$p_0(n) := \begin{cases} 3 & \text{ if } 5 \leq n \leq 8 \\ 2 & \text{ if } n \geq 9 \end{cases}$$ defined immediately before equation (4).

In [2]:
def pZero(n):
    if (n <= 8):
        return 3.0
    else:
        return 2.0

The function below corresponds to the quantity $$K_d(n) = 2\sqrt{\frac{2n}{(n-1)(n-2)}} \cdot \left(2.032^{1/n} \left(1 + \sqrt{\frac{2}{(n-2)p_0^n}}\right)\right)^d$$ defined in the statement of theorem 3.3.

In [3]:
def K(d,n):
    return 2*sqrt(float((2*n)/((n-1)*(n-2))))*(2.032**(1/n)*(1+sqrt(float(2/((n-2)* pZero(n)**n)))))**d

The function below corresponds to the quantity $$n^* = \frac{n-2}{2}$$ defined in the statement of theorem 3.3.

In [4]:
def star(n):
    return (n-2) * 0.5

The function below corresponds to the quantity $$Q_1 := \frac{p_0(n)^{n^* - d_0}}{K_{d_0}(n)}$$ defined shortly before equation (7).

In [5]:
def qOne(dZero,d,n):
    return pZero(n)**(star(n)-dZero)/K(dZero,n)

The function below checks to make sure that the choices of $d_0$ and $d$ are valid for a given value of $n$.  Namely, it checks the three conditions stated in equation (7):

\begin{align*}
    &0 \leq d_0 \leq n^* - 1.4\\
    &1 < d \leq n^*\\
    &Q_1^{d-1} > \max(1,K_d(n))
\end{align*}

In [6]:
def validSmall(dZero,d,n):
    return (0<=dZero) and (dZero <= star(n)-1.4) and (1 < d) and (d <= star(n)) and (qOne(dZero,d,n)>max(1,K(d,n)**(1/(d-1))))

The function below checks to make sure that the values $a$ and $b$ are valid given a value of $n$.  Namely, it is checking the condition given in equation (8), i.e. $$0 < a < b < 1 - \sqrt{2 \cdot \frac{n + a^2}{n^2}}$$

In [7]:
def validLarge(a,b,n):
    return (0<a) and (a < b) and (b < 1 - sqrt(float(2*(n + a**2)/n**2)))

The function below defines the quantity $L$ given shortly after equation (8), namely $$L = \frac{\sqrt{2(n + a^2)}}{1 - b}$$

In [8]:
def L(a,b,n):
    return sqrt(float(2*(n+a**2)))/(1-b)

The function below defines the quantity $D$ given shortly after equation (8), namely $$D = \frac{L}{n - L}$$

In [9]:
def D(a,b,n):
    return L(a,b,n)/(n - L(a,b,n))

The function below defines the quantity $A$ given shortly after equation (8), namely $$A = \frac{1}{a^2}$$

In [10]:
def A(a,b,n):
    return 1/a**2

The function below gives the definition of $\chi_n$ given in equation (9), namely $$\chi_n = D(A + 1) + 1$$

In [11]:
def chiN(a,b,n):
    return D(a,b,n)*(A(a,b,n) + 1)+1

The function below gives the definition of $\pi_n$ given in equation (10), namely $$\pi_n = (D(4 + A) + 2)\log(2) +  \frac{(D + 1)\log(n)}{2} + \frac{nAD}{2}$$

In [12]:
def piN(a,b,n):
    return (D(a,b,n)*(4 + A(a,b,n)) + 2)*float(log(2))+(D(a,b,n) + 1)*float(log(n))/2 + n*A(a,b,n)*D(a,b,n)/2

The function below gives the definition of $E$ given shortly after equation (8), namely $$E = \frac{1}{2(b^2 - a^2)}$$

In [13]:
def E(a,b,n):
    return 1/(2*(b**2-a**2))

The function below gives the definition of $Z$ given in equation (11), namely $$Z = \left\lfloor \frac{\log E + 2\log n - \log(L - 2)}{\log(n-1)}\right\rfloor + 2$$

In [14]:
def Z(dZero,d,a,b,n):
    return floor((float(log(E(a,b,n))) + 2*float(log(n)) - float(log(L(a,b,n) - 2)))/float(log(n-1)))+2

The function below gives the definition of $T$ given in equation (6), namely $$T = \left\lfloor\max\left(\frac{\log\left(\frac{\chi_n}{\frac{d_0(d - 1) + d}{n(d - 1)}} + 1\right)}{\log d}, \frac{\log\left(\frac{\pi_n}{\log K_d(n)^{-\frac{1}{d-1}}Q_1} + 1\right)}{\log d}\right)\right\rfloor + 2$$

In [15]:
def T(dZero,d,a,b,n):
    firstQuantity = float(log(chiN(a,b,n) * n * (d - 1) / (dZero * (d-1) + d) + 1))/float(log(d))
    secondQuantity = float(log(piN(a,b,n)/float(log(K(d,n)**(-1/(d-1))*qOne(dZero,d,n))) + 1))/float(log(d))
    return floor(max(firstQuantity,secondQuantity))+2

The function below takes in a range of degrees and a precision value.  It loops over values $n$ satisfying $n_{\text{min}} \leq n < n_{\text{max}}$ and for each $n$, it returns optimal parameter choices to minimize the sum $T+Z$.

Based on $n$, it computes an upper bound for $a$, namely the largest value of $a$ for which a valid choice of $b$ exists.  While this is not necessary, it does improve the efficiency slightly.  The value was computed by solving the equation $a = 1 - \sqrt{2 \cdot \frac{n + a^2}{n^2}}$ for $a$.

With this upper bound for $a$, the code loops over values $a \in \{\text{prec},2\cdot\text{prec},3\cdot\text{prec},\cdots,k\cdot\text{prec}\}$ where $k\cdot\text{prec} \leq a_{\text{upper}} < (k + 1)\cdot\text{prec}$.

After choosing a value of $a$, the code loops over values of $d_0 \in \{0,u,2u,3u,\cdots,ku\}$ where $u = \text{prec} \cdot (n^* - 1.4)$ and $ku \leq n^* - 1.4 < (k + 1)u$.

Next, the code loops over values of $b \in \{a + \text{prec},a + 2\cdot\text{prec},\dots,a + k\cdot\text{prec}\}$ where $a + k\cdot\text{prec} < 1 - \sqrt{2\frac{n + a^2}{n^2}} < a + (k+1)\cdot\text{prec}$.

Once the code has chosen each of the above parameters, it computes $T$ and $Z$ for the given parameters.  If $T + Z$ for the given parameters is smaller than what is stored in $\text{tempMinT} + \text{tempMinZ}$, the code sets $\text{tempMinT} = T$ and $\text{tempMinZ} = Z$, then checks to make sure that the parameters are valid and satisfy the necessary hypotheses in Proposition 3.2 before storing the parameters $a,b,$ and $d_0$.  After checking all values of $a,b,$ and $d_0$, the code stores the optimal parameter values along with the value of $n$ in an array.

The code returns an array of the form \[\[$n_{\text{min}},d_0,d,a,b,T,Z$\],\[$n_{\text{min}}+1,d_0,d,a,b,T,Z$\],...,\[$n_{\text{max}}-1,d_0,d,a,b,T,Z$\]\] where $d$ is always chosen to be $n^*$ and for any given $n$ satisfying $n_{\text{min}} \leq n < n_{\text{max}}$, the printed values of $d_0,a,b$ minimize the sum $T + Z$.

A few miscellaneous comments:
- The minimal possible values of $T$ and $Z$ are $2$.  So if $T + Z = 4$ at any point in the code, we can assume that is the minimal sum and there is no need to check for further values of $a,b,d_0$
- It seems like the value $d = n^*$ always allows for $T + Z$ to be minimal.  Hence, we do not bother looping over different values of $d$ for efficiency.  That said, one can easily modify the code to loop over different values of $d$ too in exchange for longer run time.

In [16]:
def minTPlusZ(nMin,nMax,prec):
    toReturn = []
    for n in range(nMin,nMax):
        nStar = star(n)
        aUpper = (2*n**2 - sqrt(4*n**4 - 4*(n**2 - 2*n)*(n**2 - 2)))/(2*(n**2 - 2))
        a = prec
        tempMinA=prec
        dZero = 0
        tempMinDZero = 0
        b = a + prec
        tempMinB = a + prec
        tempMinT = T(dZero,nStar,a,b,n)
        tempMinZ = Z(dZero,nStar,a,b,n)
        tempMinSum = tempMinT + tempMinZ
        while ((a <= aUpper) and (tempMinSum > 4)):
            while ((dZero <= nStar - 1.4) and (tempMinSum > 4)):
                while ((b < 1-sqrt(2*(n + a**2)/n**2)) and (tempMinSum > 4)):
                    tempT = T(dZero,nStar,a,b,n)
                    tempZ = Z(dZero,nStar,a,b,n)
                    if (tempT + tempZ < tempMinSum):
                        tempMinA = a
                        tempMinB = b
                        tempMinDZero = dZero
                        tempMinT = tempT
                        tempMinZ = tempZ
                        tempMinSum = tempT + tempZ
                    b += prec
                dZero += prec * (nStar - 1.4)
                b = a + prec
            a += prec
            dZero = 0
            b = a + prec
        assert validSmall(tempMinDZero,nStar,n), "d0,d,n are invalid"
        assert validLarge(tempMinA,tempMinB,n), "a,b,n are invalid"
        assert chiN(tempMinA,tempMinB,n) >= 2, "chiN is too small"
        assert piN(tempMinA,tempMinB,n) >= 5*log(2) + 2*log(n), "piN is too small"
        toReturn.append([n,tempMinDZero,nStar,tempMinA,tempMinB,tempMinT,tempMinZ])
    return toReturn

The idea of the program below is to take a maximal value of $n$ as input, check to see if there exist choices of $a,b,d_0$ so that $T + Z = 4$.  If so, decrement $n$ by $1$ and then try again.  Continue until it is no longer possible to find choices of $a,b,d_0$ so that $T + Z = 4$.

In more detail, the program loops over values of $n \leq n_{\text{max}}$ for which $T+Z = 4$.  Given $n$, the program defines the same upper bound on $a$ as above (see the commentary on the previous function).  From there, it loops over values of $a \in \{a_{\text{upper}} - \text{prec},a_{\text{upper}} - 2\cdot\text{prec},\dots,a_{\text{upper}} - k\cdot\text{prec}\}$ where $a_{\text{upper}} - k\cdot\text{prec} > 0 \geq a_{\text{upper}} - (k+1)\cdot\text{prec}$.

Next, the program loops over $b \in \{a_{\text{upper}},a_{\text{upper}} - \text{prec},a_{\text{upper}} - 2\cdot\text{prec},\dots,a_{\text{upper}} - k\cdot\text{prec}\}$ where $a_{\text{upper}} - k\cdot\text{prec} > a \geq a_{\text{upper}} - (k+1)\cdot\text{prec}$.

Next, the program loops over values of $d_0 \in \{n^* - 1.4, n^* - 1.4 - u, n^* - 1.4 - 2u,\dots,n^* - 1.4 - ku\}$ where $u = \text{prec}\cdot(n^* - 1.4)$ and $n^* - 1.4 - ku \geq 0 > n^* - 1.4 - (k+1)u$.

Upon choosing values for $a,b,d_0$, the program uses $d = n^*$ and computes the values of $T$ and $Z$ at these parameters.  If $T$ and $Z$ are minimal (equivalently, if $T + Z = 4$), the program checks to make sure that the parameters are valid and that the hypotheses of Proposition 3.2 are met before saving the values of $a,b,d_0,d$, then decrementing $n$ by 1 and trying again.  If the program cannot find values of $a,b,d_0$ to obtain $T + Z = 4$, it terminates.

The program returns a double ended array (deque) of the form \[\[$n_{\text{min}},d_0,d,a,b,T,Z$\],\[$n_{\text{min}}+1,d_0,d,a,b,T,Z$\],...,\[$n_{\text{max}},d_0,d,a,b,T,Z$\]\] where $n_{\text{min}}$ is the smallest value of $n$ for which the program could find parameters to get $T = Z = 2$ and for each $n$ satisfying $n_{\text{min}} \leq n \leq n_{\text{max}}$, the values of $d_0,d,a,b$ yield $T = Z = 2$.

Additional comments:
- Most of the comments that apply to the above program also apply here.
- This program initializes $b = a_{\text{upper}}$ because by coincidence, this happens to find additional values of $n$ for which $T = Z = 2$.  The next program ("minNWithMinTPlusZAlt") starts with a slightly larger value of $b$.
- This program begins with maximal possible values of $a,b,d_0$ because these larger values tend to produce smaller values of $T$ and $Z$.

In [17]:
def minNWithMinTPlusZ(nMax,prec):
    n = nMax
    nStar = star(n)
    aUpper = (2*n**2 - sqrt(4*n**4 - 4*(n**2 - 2*n)*(n**2 - 2)))/(2*(n**2 - 2))
    a = aUpper - prec
    b = aUpper
    dZero = nStar - 1.4
    tempTPlusZ = 4
    listOfParams = deque([])
    while ((n >= 6) and (tempTPlusZ == 4)):
        tempT = T(dZero,nStar,a,b,n)
        tempZ = Z(dZero,nStar,a,b,n)
        tempTPlusZ = tempT + tempZ
        while ((a > 0) and (tempTPlusZ > 4)):
            while ((b > a) and (tempTPlusZ > 4)):
                while ((dZero >= 0) and (tempTPlusZ > 4)):
                    tempT = T(dZero,nStar,a,b,n)
                    tempZ = Z(dZero,nStar,a,b,n)
                    tempTPlusZ = tempT + tempZ
                    if (tempTPlusZ == 4):
                        assert validSmall(dZero,nStar,n), "d0,d,n are invalid"
                        assert validLarge(a,b,n), "a,b,n are invalid"
                        assert chiN(a,b,n) >= 2, "chiN is too small"
                        assert piN(a,b,n) >= 5*log(2) + 2*log(n), "piN is too small"
                        listOfParams.appendleft([n,dZero,nStar,a,b,tempT,tempZ])
                    dZero -= prec*(nStar - 1.4)
                b -= prec
                dZero = nStar - 1.4
            a -= prec
            b = aUpper
        n -= 1
        aUpper = (2*n**2 - sqrt(4*n**4 - 4*(n**2 - 2*n)*(n**2 - 2)))/(2*(n**2 - 2))
        nStar = star(n)
        a = aUpper - prec
        b = aUpper
        dZero = nStar - 1.4
    return listOfParams

This program is identical to the above program with a slightly different initialization of $b$.  See the program immediately above this for comments.

In [18]:
def minNWithMinTPlusZAlt(nMax,prec):
    n = nMax
    nStar = star(n)
    aUpper = (2*n**2 - sqrt(4*n**4 - 4*(n**2 - 2*n)*(n**2 - 2)))/(2*(n**2 - 2))
    a = aUpper - prec
    b = 1 - sqrt(2*(n+a**2)/n**2)-prec
    dZero = nStar - 1.4
    tempTPlusZ = 4
    listOfParams = deque([])
    while ((n >= 6) and (tempTPlusZ == 4)):
        tempT = T(dZero,nStar,a,b,n)
        tempZ = Z(dZero,nStar,a,b,n)
        tempTPlusZ = tempT + tempZ
        while ((a > 0) and (tempTPlusZ > 4)):
            while ((b > a) and (tempTPlusZ > 4)):
                while ((dZero >= 0) and (tempTPlusZ > 4)):
                    tempT = T(dZero,nStar,a,b,n)
                    tempZ = Z(dZero,nStar,a,b,n)
                    tempTPlusZ = tempT + tempZ
                    if (tempTPlusZ == 4):
                        assert validSmall(dZero,nStar,n), "d0,d,n are invalid"
                        assert validLarge(a,b,n), "a,b,n are invalid"
                        assert chiN(a,b,n) >= 2, "chiN is too small"
                        assert piN(a,b,n) >= 5*log(2) + 2*log(n), "piN is too small"
                        listOfParams.appendleft([n,dZero,nStar,a,b,tempT,tempZ])
                    dZero -= prec*(nStar - 1.4)
                b -= prec
                dZero = nStar - 1.4
            a -= prec
            b = 1 - sqrt(2*(n+a**2)/n**2)-prec
        n -= 1
        aUpper = (2*n**2 - sqrt(4*n**4 - 4*(n**2 - 2*n)*(n**2 - 2)))/(2*(n**2 - 2))
        nStar = star(n)
        a = aUpper - prec
        b = 1 - sqrt(2*(n+a**2)/n**2)-prec
        dZero = nStar - 1.4
    return listOfParams

# Computations

These computations verify Lemma 3.2 for $n \leq 506$.

In [19]:
minTPlusZ(6,219,0.01)

[[6, 0, 2.0, 0.18000000000000002, 0.2900000000000001, 10, 4],
 [7, 0.5500000000000003, 2.5, 0.20000000000000004, 0.2800000000000001, 7, 4],
 [8, 0.9920000000000008, 3.0, 0.16, 0.4100000000000002, 7, 3],
 [9, 0.8820000000000005, 3.5, 0.17, 0.4000000000000002, 6, 3],
 [10, 1.1960000000000008, 4.0, 0.23000000000000007, 0.4100000000000002, 5, 3],
 [11, 1.7049999999999987, 4.5, 0.13999999999999999, 0.37000000000000016, 5, 3],
 [12, 2.0880000000000014, 5.0, 0.2700000000000001, 0.4100000000000002, 4, 3],
 [13, 2.254999999999998, 5.5, 0.20000000000000004, 0.37000000000000016, 4, 3],
 [14, 2.529999999999999, 6.0, 0.16, 0.35000000000000014, 4, 3],
 [15, 3.009000000000002, 6.5, 0.12999999999999998, 0.34000000000000014, 4, 3],
 [16, 3.1360000000000023, 7.0, 0.10999999999999999, 0.3200000000000001, 4, 3],
 [17, 3.9649999999999967, 7.5, 0.3300000000000001, 0.4300000000000002, 3, 3],
 [18, 4.289999999999996, 8.0, 0.2700000000000001, 0.3900000000000002, 3, 3],
 [19, 4.402000000000003, 8.5, 0.240000000

In [20]:
minNWithMinTPlusZ(507,0.01)

deque([[223,
        69.82400000000021,
        110.5,
        0.40512345713847286,
        0.8851234571384733,
        2,
        2],
       [224,
        70.14399999999986,
        111.0,
        0.40533616439322917,
        0.8853361643932296,
        2,
        2],
       [225,
        70.46400000000003,
        111.5,
        0.40554744794423947,
        0.8855474479442399,
        2,
        2],
       [226,
        70.78400000000019,
        112.0,
        0.40575732359838645,
        0.8857573235983869,
        2,
        2],
       [227,
        72.21499999999985,
        112.5,
        0.40596580691799933,
        0.8859658069179998,
        2,
        2],
       [228, 72.54, 113.0, 0.40617291322569704, 0.8861729132256975, 2, 2],
       [229,
        72.86500000000017,
        113.5,
        0.40637865760911374,
        0.8863786576091142,
        2,
        2],
       [230,
        74.31599999999983,
        114.0,
        0.40658305492550995,
        0.8865830549255104,
   

In [21]:
minNWithMinTPlusZ(222,0.001)

deque([[219,
        68.22269999999904,
        108.5,
        0.39925806610361225,
        0.8832580661036127,
        2,
        2],
       [220,
        68.64879999999818,
        109.0,
        0.40847663130953416,
        0.8844766313095346,
        2,
        2],
       [221,
        69.29210000000243,
        109.5,
        0.40869370703753083,
        0.8846937070375313,
        2,
        2],
       [222,
        69.61260000000158,
        110.0,
        0.4179093101235729,
        0.8859093101235733,
        2,
        2]])